# CNN implementation

In [ ]:
import os

#os.chdir("/drive/MyDrive/data")

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from torchvision.transforms import Compose, ToTensor, RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter
import torch

import random
import utils

plt.rcParams['figure.figsize'] = (17, 5)

In [2]:
def saveheavy(df, name, n):  #functions to save havy dataframes in multiple files
    l= len(df)
    for i in range(n-1):
        df.iloc[int((l/n)*i):int((l/n)*(i+1))].to_hdf(f'{name}_{i+1}.h5', 'x', mode='w')
    df.iloc[int((l/n)*(n-1)):l].to_hdf(f'{name}_{n}.h5', 'x', mode='w')
    
def readheavy(name, n, column):
    result = pd.DataFrame(columns=column)
    for i in range(n):
        df = pd.read_hdf(f'Data/MelMfcc/{name}_{i+1}.h5', 'x')
        print(i)
        result = pd.concat([result, df], ignore_index=True)
    return result

In [3]:
#Restore Datasets

test = pd.read_hdf('Data/MelMfcc/test.h5', 'x')
validation = pd.read_hdf('Data/MelMfcc/validation.h5', 'x')
training = readheavy('training', 8, ['mel', 'mfcc', 'y'])

HDF5ExtError: HDF5 error back trace

  File "C:\ci\hdf5_1655187604574\work\src\H5Dio.c", line 199, in H5Dread
    can't read data
  File "C:\ci\hdf5_1655187604574\work\src\H5Dio.c", line 603, in H5D__read
    can't read data
  File "C:\ci\hdf5_1655187604574\work\src\H5Dchunk.c", line 2316, in H5D__chunk_read
    chunked read failed
  File "C:\ci\hdf5_1655187604574\work\src\H5Dscatgath.c", line 550, in H5D__scatgath_read
    datatype conversion failed
  File "C:\ci\hdf5_1655187604574\work\src\H5T.c", line 5025, in H5T_convert
    datatype conversion failed
  File "C:\ci\hdf5_1655187604574\work\src\H5Tconv.c", line 3221, in H5T__conv_vlen
    memory allocation failed for type conversion
  File "C:\ci\hdf5_1655187604574\work\src\H5FL.c", line 922, in H5FL_blk_malloc
    memory allocation failed for chunk
  File "C:\ci\hdf5_1655187604574\work\src\H5FL.c", line 250, in H5FL_malloc
    memory allocation failed for chunk

End of HDF5 error back trace

VLArray._read_array: Problems reading the

In [ ]:
from torch.utils.data import Dataset, DataLoader

#Class for the creation of torch manageble datasets, with Format one can select the desired input column 
class DataAudio(Dataset):

    def __init__(self, split, Format, transform=None):
        self.x = split[Format]
        self.y = split['y']
        self.transform = transform

    def __len__(self):
        return min(32000, len(self.x))
        #return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx, 0, 0].astype(float)
        if self.transform:
            x = self.transform(x)
        return x, y

In [ ]:
transforms = Compose([
    ToTensor(), #this converts numpy or Pil image to torch tensor and normalizes it in 0, 1
    RandomAffine((0.05, 0.05)),
    RandomHorizontalFlip(),
    RandomVerticalFlip()
])

In [ ]:
#Creation of torch suited dataset classes  (change string 'mel' to select desired Format)
training_dataset = DataAudio(training, 'mel',transforms)
validation_dataset = DataAudio(validation, 'mel',transforms)
test_dataset = DataAudio(test, 'mel',transforms)

In [ ]:
#Creation of dataloader classes
batch_size = 64
training_dataloader = DataLoader(training_dataset, batch_size, shuffle=False, num_workers=os.cpu_count())
validation_dataloader = DataLoader(validation_dataset, batch_size, shuffle=False, num_workers=os.cpu_count())
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=os.cpu_count())